In [1]:
!pip install peft
!pip install evaluate

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import wandb

from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np
import pandas as pd
import random

In [3]:
# model_name = "tiiuae/falcon-7b"
# model_name = "tiiuae/falcon-rw-1b"
model_name = "distilbert-base-uncased"
# mistral = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

C:\Users\DELL 5540\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
# define label maps
id2label = {0: "Irrelevant", 1: "Relevant"}
label2id = {"Irrelevant":0, "Relevant":1}

# generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Tokenize input text
input_text = "Hello"
inputs = tokenizer(input_text, return_tensors="pt")

# Inference
outputs = model(**inputs)

# Post-processing
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1).item()
print("Predicted Class:", predicted_class)

Predicted Class: 1


# Low Rank Adaptation Fine Tuning

## Dataset Gathering

In [7]:
df = pd.read_csv('C:\\Users\\DELL 5540\\Desktop\\few-shot-learning-LLM-for-irrelevance-classification-main\\data\\WaTA_dataset.csv', encoding = "ISO-8859-1")

test_set = 0.3

x_train, y_train, x_test, y_test = [], [], [] ,[]
relevant_sentences, irrelevant_sentences = [], []

for index, row in df.iterrows():
    if row['Class'] == 'Relevant':
        relevant_sentences.append(row['Sentence'])
    else:
        irrelevant_sentences.append(row['Sentence'])

random.shuffle(relevant_sentences)
random.shuffle(irrelevant_sentences)

relevant_train = relevant_sentences[:int(len(relevant_sentences)*(1-test_set))]
relevant_test = relevant_sentences[int(len(relevant_sentences)*(1-test_set)):]
irrelevant_train = irrelevant_sentences[:int(len(irrelevant_sentences)*(1-test_set))]
irrelevant_test = irrelevant_sentences[int(len(irrelevant_sentences)*(1-test_set)):]

for sentence in relevant_train:
    x_train.append(sentence)
    y_train.append(1)
    
for sentence in irrelevant_train:
    x_train.append(sentence)
    y_train.append(0)

for sentence in relevant_test:
    x_test.append(sentence)
    y_test.append(1)
    
for sentence in irrelevant_test:
    x_test.append(sentence)
    y_test.append(0)       
        
dataset = DatasetDict({'train':Dataset.from_dict({'label':y_train,'text':x_train}),
                        'test':Dataset.from_dict({'label':y_test,'text':x_test})})

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 17577
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 7534
    })
})

## Preprocess data

In [9]:
# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [10]:
# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["text"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=2048
    )

    return tokenized_inputs

In [11]:
# tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/17577 [00:00<?, ? examples/s]

Map:   0%|          | 0/7534 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 17577
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 7534
    })
})

In [12]:
# create data collator (similar to a pytorch dataloader)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Evaluation

In [13]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

In [14]:
# define an evaluation function to pass into trainer later
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels), 
            "precision": precision.compute(predictions=predictions, references=labels),
            "recall": recall.compute(predictions=predictions, references=labels),
            "f1": f1.compute(predictions=predictions, references=labels)}

In [15]:

def evaluate_model(examples):
    correct = 0
    for sentence in examples:
        # tokenize text
        inputs = tokenizer.encode(sentence["text"], return_tensors="pt").to(model.device)
        # compute logits
        logits = model(inputs).logits
        # convert logits to label
        predictions = torch.argmax(logits)
        
        if predictions.item() == sentence["label"]:
            correct += 1

        print(sentence["text"])
        print("Predicted:", id2label[predictions.item()] + ", Actual:", id2label[sentence["label"]])
    print("Accuracy:", correct/len(examples))

In [16]:
# define list of examples
examples = dataset["test"].shuffle().select(range(10))

print("Untrained model predictions:")
print("----------------------------")
evaluate_model(examples)

Untrained model predictions:
----------------------------
b Any missing data is collected by telephone the sales department is still responsible 
Predicted: Relevant, Actual: Relevant
If for some reason the day proposed by the transporter does not suit the client the financial department will analyze if it could economically be possible to charter a truck just for him
Predicted: Relevant, Actual: Relevant
If the design fails the test then it is sent back to the first Activity in the department to review and create a new Update Plan
Predicted: Irrelevant, Actual: Relevant
The kitchen is open therefore the customers can see everything
Predicted: Relevant, Actual: Irrelevant
The beads are then fed into a mold of the desired shape 
Predicted: Relevant, Actual: Relevant
If the parents don t agree they still have to send a message and there is no inscription
Predicted: Relevant, Actual: Relevant
The alkaline solution removes the remaining film of oil and leaves a surface ready for acid pickl

# Train model

In [17]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['q_lin', 'v_lin'])

In [18]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 665,858 || all params: 67,620,868 || trainable%: 0.9846930684178736


In [19]:
# hyperparameters
lr = 1e-3
batch_size = 16
num_epochs = 10
weight_decay=0.01

In [20]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_name + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=weight_decay,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [21]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

C:\Users\DELL 5540\AppData\Roaming\Python\Python311\site-packages\accelerate\accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [22]:
# # train model
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: maherwshwshny (mlfortm). Use `wandb login --relogin` to force relogin


  0%|          | 0/10990 [00:00<?, ?it/s]

{'loss': 0.3149, 'grad_norm': 1.8673523664474487, 'learning_rate': 0.0009545040946314831, 'epoch': 0.45}
{'loss': 0.2904, 'grad_norm': 6.483707427978516, 'learning_rate': 0.0009090081892629664, 'epoch': 0.91}


  0%|          | 0/471 [00:00<?, ?it/s]

Trainer is attempting to log a value of "{'accuracy': 0.8841252986461375}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9358674112772474}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.9094887955182073}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9224895676107608}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


...

Trainer is attempting to log a value of "{'accuracy': 0.8865144677462171}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9156255348279994}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.936624649859944}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.926006057983557}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.5102741122245789, 'eval_accuracy': {'accuracy': 0.8865144677462171}, 'eval_precision': {'precision': 0.9156255348279994}, 'eval_recall': {'recall': 0.936624649859944}, 'eval_f1': {'f1': 0.926006057983557}, 'eval_runtime': 170.3963, 'eval_samples_per_second': 44.215, 'eval_steps_per_second': 2.764, 'epoch': 10.0}


C:\Users\DELL 5540\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'train_runtime': 8497.4261, 'train_samples_per_second': 20.685, 'train_steps_per_second': 1.293, 'train_loss': 0.20403827288890558, 'epoch': 10.0}


TrainOutput(global_step=10990, training_loss=0.20403827288890558, metrics={'train_runtime': 8497.4261, 'train_samples_per_second': 20.685, 'train_steps_per_second': 1.293, 'train_loss': 0.20403827288890558, 'epoch': 10.0})

In [23]:
evaluate_model(examples)

b Any missing data is collected by telephone the sales department is still responsible 
Predicted: Relevant, Actual: Relevant
If for some reason the day proposed by the transporter does not suit the client the financial department will analyze if it could economically be possible to charter a truck just for him
Predicted: Relevant, Actual: Relevant
If the design fails the test then it is sent back to the first Activity in the department to review and create a new Update Plan
Predicted: Relevant, Actual: Relevant
The kitchen is open therefore the customers can see everything
Predicted: Irrelevant, Actual: Irrelevant
The beads are then fed into a mold of the desired shape 
Predicted: Relevant, Actual: Relevant
If the parents don t agree they still have to send a message and there is no inscription
Predicted: Relevant, Actual: Relevant
The alkaline solution removes the remaining film of oil and leaves a surface ready for acid pickling
Predicted: Relevant, Actual: Relevant
Allow to dry for

# Save fine-tuned model

In [24]:
peft_model_id = "distilbert-base-uncased-lora-text-classification"
trainer.save_model(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

C:\Users\DELL 5540\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('distilbert-base-uncased-lora-text-classification\\tokenizer_config.json',
 'distilbert-base-uncased-lora-text-classification\\special_tokens_map.json',
 'distilbert-base-uncased-lora-text-classification\\vocab.txt',
 'distilbert-base-uncased-lora-text-classification\\added_tokens.json',
 'distilbert-base-uncased-lora-text-classification\\tokenizer.json')

In [25]:
config = PeftConfig.from_pretrained(peft_model_id)

test_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, id2label=id2label, label2id=label2id)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model_inf = PeftModel.from_pretrained(test_model, peft_model_id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
evaluate_model(examples)

b Any missing data is collected by telephone the sales department is still responsible 
Predicted: Relevant, Actual: Relevant
If for some reason the day proposed by the transporter does not suit the client the financial department will analyze if it could economically be possible to charter a truck just for him
Predicted: Relevant, Actual: Relevant
If the design fails the test then it is sent back to the first Activity in the department to review and create a new Update Plan
Predicted: Relevant, Actual: Relevant
The kitchen is open therefore the customers can see everything
Predicted: Irrelevant, Actual: Irrelevant
The beads are then fed into a mold of the desired shape 
Predicted: Relevant, Actual: Relevant
If the parents don t agree they still have to send a message and there is no inscription
Predicted: Relevant, Actual: Relevant
The alkaline solution removes the remaining film of oil and leaves a surface ready for acid pickling
Predicted: Relevant, Actual: Relevant
Allow to dry for